<a href="https://colab.research.google.com/github/manish284/Natural-Language-Processing/blob/master/Sentence_Semantic_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem Statement:

In attached data there are various headlines (Sentences) from various newspapers. 
Your task is to build a model that can search the similar sentences from the data set.

for example: "where are you currently residing?" is similar to "what is your address?"

if you pass query = " where are you currently residing ?"
Your model should be able to return like something below:

1. What is your current address?
2. Can you give me your current address?
3. Please provide me your address.
4. In which city you residing ............... and so on

The most similar sentence should be on top and follow the order.

#importing the required packages

In [56]:
import numpy as np
import pandas as pd
import gensim
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.corpus import stopwords

In [57]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## unzip the data folder

In [3]:
!unzip /content/news-text.csv.zip

Archive:  /content/news-text.csv.zip
  inflating: news-text.csv           


#read the file data

In [58]:
dataframe = pd.read_csv('/content/news-text.csv')

In [59]:
#check the top 5 row
print(dataframe.head())
dataframe.shape

   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
4      20030219      air nz strike to affect australian travellers


(1186018, 2)

In [60]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
GOOD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [61]:
def text_prepare(text):
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = GOOD_SYMBOLS_RE.sub('', text)
    text = ' '.join([x for x in text.split() if x and x not in STOPWORDS])
    return text.strip()

In [62]:
prepared_dataframe = []
for line in dataframe['headline_text']:
  prepared_dataframe.append(text_prepare(line))

In [63]:
prepared_dataframe[0:10]

['aba decides community broadcasting licence',
 'act fire witnesses must aware defamation',
 'g calls infrastructure protection summit',
 'air nz staff aust strike pay rise',
 'air nz strike affect australian travellers',
 'ambitious olsson wins triple jump',
 'antic delighted record breaking barca',
 'aussie qualifier stosur wastes four memphis match',
 'aust addresses un security council iraq',
 'australia locked war timetable opp']

#downloading Pre-trained word and phrase vectors
pre-trained vectors trained on part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases.

In [64]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

--2020-10-11 20:06:29--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.77.102
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.77.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz.1’

GoogleNews-vectors- 100%[===================>]   1.53G  94.1MB/s    in 42s     

2020-10-11 20:07:12 (37.2 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz.1’ saved [1647046227/1647046227]



In [65]:
wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True, limit=500000)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# hecking Pre-trained word vectors
wv_embeddings['me']

In [66]:
def question_to_vec(question, embeddings, dim=300):
    """
        question: a string
        embeddings: dict where the key is a word and a value is its' embedding
        dim: size of the representation

        result: vector representation for the question
    """
    result = np.zeros(dim)
    c = 0
    words = question.split()
    for word in words:
        if word in embeddings:
            result += np.array(embeddings[word])
            c += 1
    if c != 0:
        result /= c
    return result

In [67]:
def rank_candidates(question, candidates, embeddings, dim=300):
    """
        question: a string
        candidates: a list of strings (candidates) which we want to rank
        embeddings: some embeddings
        dim: dimension of the current embeddings
        
        result: a list of pairs (initial position in the list, question)
    """
    q_vecs = np.array([question_to_vec(question, embeddings, dim) for i in range(len(candidates))])
    cand_vecs = np.array([question_to_vec(candidate, embeddings, dim) for candidate in candidates])
    cosines = np.array(cosine_similarity(q_vecs, cand_vecs)[0])
    merged_list = list(zip(cosines, range(len(candidates)), candidates))
    sorted_list  = sorted(merged_list, key=lambda x: x[0], reverse=True)
    result = [(a,b,c) for a,b,c in sorted_list if a> 0.4]
    return result

In [90]:
wv_prepared_ranking = [] 
for indx,question in enumerate(prepared_dataframe[0:100]):
    candidate=prepared_dataframe[0:100]
    ranks = rank_candidates(question, candidate, wv_embeddings)
    wv_prepared_ranking.append([r[2] for r in ranks])

In [69]:
wv_prepared_ranking[0:2]

[['aba decides community broadcasting licence',
  'community urged help homeless youth',
  'council chief executive fails secure position',
  'council welcomes ambulance levy decision'],
 ['act fire witnesses must aware defamation',
  'bushfire victims urged see centrelink',
  'dargo fire threat expected rise',
  'jury consider verdict murder case',
  'businesses prepare terrorist attacks',
  'griffiths fire project knock back',
  'brigadier dismisses reports troops harassed',
  'firefighters contain acid spill']]

In [79]:
def prepare_file():
  out = open('/content/prepared_train.tsv', 'w')
  for line in wv_prepared_ranking:
    new_line = [text_prepare(q) for q in line]
    print(*new_line, sep='\t', file=out)
  out.close()

In [80]:
prepare_file()

In [82]:
def setup_starspace():
    if not os.path.exists("/usr/local/bin/starspace"):
        os.system("wget https://dl.bintray.com/boostorg/release/1.63.0/source/boost_1_63_0.zip")
        os.system("unzip boost_1_63_0.zip && mv boost_1_63_0 /usr/local/bin")
        os.system("git clone https://github.com/facebookresearch/Starspace.git")
        os.system("cd Starspace && make && cp -Rf starspace /usr/local/bin")

In [84]:
import os
setup_starspace()

In [85]:
!starspace train -trainFile "/content/prepared_train.tsv" -model starspace_embedding \
-trainMode 3 -adagrad true -ngrams 1 -epoch 5 -dim 100 -similarity cosine -minCount 2 \
-verbose true -fileFormat labelDoc -negSearchLimit 10 -lr 0.05

Arguments: 
lr: 0.05
dim: 100
epoch: 5
maxTrainTime: 8640000
validationPatience: 10
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 10
batchSize: 5
thread: 10
minCount: 2
minCountLabel: 1
label: __label__
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 3
fileFormat: labelDoc
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
useWeight: 0
weightSep: :
Start to initialize starspace model.
Build dict from input file : /content/prepared_train.tsv
Read 0M words
Number of words in dictionary:  394
Number of labels in dictionary: 0
Loading data from file : /content/prepared_train.tsv
Total number of examples loaded : 89
Initialized model weights. Model size :
matrix : 394 100
Training epoch 0: 0.05 0.01
Epoch: 88.9%  lr: 0.050000  loss: 0.332594  eta: <1min   tot: 0h0m0s  (17.8%)
 ---+++                Epoch    0 Train error : 0.61408126 +++--- ☃
Training epoch 1: 0.04 0.01
Epoch: 88.9%  lr: 0.040000  loss: 0.188699  eta: <1min   tot: